In [1]:
import pandas as pd
# import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('data/clean_listings.csv')
df = df.drop('id', axis=1)

In [3]:
# drop les lignes avec batrooms = nan
df = df.dropna(subset=['bathrooms'])
print(df.isnull().sum())

neighbourhood       0
room_type           0
accommodates        0
bathrooms           0
bedrooms            0
beds                0
amenities           0
price               0
availability_365    0
dtype: int64


In [4]:
# Diviser les données en ensembles d'entraînement et de test
y = df['price']
X = df.drop(['price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.2)

In [5]:
# Identifier les colonnes numériques et catégoriques
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Créer un préprocesseur pour transformer les colonnes
column_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [7]:
def print_model_evaluation(y_pred, y_true):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print({ 'mse': mse, 'mae': mae, 'r2': r2 })

In [ ]:
# # Modèle Random Forest
# rf = RandomForestRegressor(random_state=42)

# # Pipeline avec le modèle Random Forest
# pipeline = Pipeline(steps=[
#     ('preprocessor', column_transformer),
#     ('model', rf)
# ])

# # Recherche des meilleurs hyperparamètres (Grid Search)
# param_grid = {
#     'model__n_estimators': [50, 100, 200],
#     'model__max_depth': [10, 20, 30, None]
# }

# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')
# grid_search.fit(X_train, y_train)

# # Meilleur modèle
# pipeline = grid_search.best_estimator_

# # Prédictions
# y_pred = pipeline.predict(X_test)

# # Évaluation
# print_model_evaluation(y_pred, y_test)

In [ ]:
# Pipeline avec Random Forest
pipeline = Pipeline(steps=[
    ('preprocessor', column_transformer),
    ('model', RandomForestRegressor(random_state=42, n_estimators=200, max_depth=20))
])

# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = pipeline.predict(X_test)

In [8]:
print_model_evaluation(y_pred, y_test)

{'mse': 5802.8577905267575, 'mae': 53.458200909057, 'r2': 0.45665935725855245}


In [9]:
# export du modèle
import joblib
joblib.dump(pipeline, 'models/random_forest.joblib')

['models/random_forest.joblib']